In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=ee1c8403e760d9d2384cb1cb8ccaa4f124abaede5fd2e0ebc72487a4225f4047
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.12 ,random_state=0 , l1_ratio=0.81, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_16_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10, 20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=336455, Sat Apr 18 14:16:38 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801812808110249e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.2
termination on tolfun=1e-11 (Sat Apr 18 14:16:38 2020)
final/bestever f-value = -1.801813e+02 -1.801813e+02
incumbent solution: [-4.90623624  2.85548689  1.21738341  1.80062768  2.70059993  4.29280509
  0.48960379  0.30405981 ...]
std deviations: [2.43708994 2.43733215 2.43735087 2.43692631 2.43684339 2.43723728
 2.43729024 2.4370855  ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=290714, Sat Apr 18 14:16:38 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801146458141778e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.11464581] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


termination on tolfun=1e-11 (Sat Apr 18 14:16:38 2020)
final/bestever f-value = -1.801146e+02 -1.801146e+02
incumbent solution: [-4.99992093  2.60569565  2.69239326 -0.62407205  3.62673171  4.38109326
 -0.13438416 -3.25332263 ...]
std deviations: [2.43689906 2.43650448 2.43687368 2.43685195 2.43674686 2.43693358
 2.43705419 2.43690665 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=285313, Sat Apr 18 14:16:38 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.802077101406580e+02 1.0e+00 2.44e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.20224273] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 -1.802077677612255e+02 1.0e+00 2.38e+00  2e+00  2e+00 0:00.4


...

    3     57 -1.802077485944120e+02 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   34    646 -1.802076677206786e+02 1.1e+00 1.96e+00  2e+00  2e+00 0:03.3
   74   1406 -1.802077151278011e+02 1.2e+00 2.04e+00  2e+00  2e+00 0:07.3
  100   1900 -1.802077306017064e+02 1.3e+00 2.13e+00  2e+00  2e+00 0:09.9
  161   3059 -1.802075979166255e+02 1.4e+00 2.18e+00  2e+00  2e+00 0:15.9
  200   3800 -1.802077108149228e+02 1.4e+00 2.26e+00  2e+00  2e+00 0:19.8
  282   5358 -1.802077528425334e+02 1.5e+00 1.95e+00  2e+00  2e+00 0:27.8
  300   5700 -1.802077803564350e+02 1.5e+00 1.93e+00  2e+00  2e+00 0:29.5
  400   7600 -1.802077610872359e+02 1.5e+00 1.89e+00  2e+00  2e+00 0:39.3
  500   9500 -1.802077681493426e+02 1.6e+00 2.04e+00  2e+00  2e+00 0:49.1
  600  11400 -1.802077793479093e+02 1.6e+00 1.81e+00  2e+00  2e+00 0:58.9
  700  13300 -1.802077727433698e+02 1.7e+00 1.92e+00  2e+00  2e+00 1:08.6
  800  15200 -1.802075736582709e+02 1.7e+00 1.97e+00  2e+00  2e+00 1:18.3
  900  17100 -1.802077399591279e+02 1.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.11464581] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


termination on tolfun=1e-11 (Sat Apr 18 16:56:54 2020)
final/bestever f-value = -1.801146e+02 -1.801146e+02
incumbent solution: [-4.68900107 -4.88827449  3.32170928  4.99996405  4.87448688 -2.41176655
  3.37658505 -4.70450107 ...]
std deviations: [2.43440845 2.43433679 2.43421786 2.43398294 2.43430666 2.43422465
 2.43427792 2.43435597 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=304519, Sat Apr 18 16:56:54 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 -1.801765470684032e+02 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 -1.801854935180064e+02 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.15983638] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.16934371] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 -1.801839994769499e+02 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   34    646 -1.802077733214437e+02 1.1e+00 1.89e+00  2e+00  2e+00 0:03.4
   76   1444 -1.802077758446967e+02 1.2e+00 1.81e+00  2e+00  2e+00 0:07.4
  100   1900 -1.802077778917139e+02 1.2e+00 1.84e+00  2e+00  2e+00 0:09.8
  162   3078 -1.802077764611001e+02 1.3e+00 1.80e+00  2e+00  2e+00 0:15.9
  200   3800 -1.802077735148255e+02 1.4e+00 1.66e+00  2e+00  2e+00 0:19.6
  282   5358 -1.802077682091914e+02 1.4e+00 1.56e+00  1e+00  2e+00 0:27.7
  300   5700 -1.802077797036391e+02 1.4e+00 1.60e+00  1e+00  2e+00 0:29.5
  400   7600 -1.802077354775476e+02 1.5e+00 1.51e+00  1e+00  2e+00 0:39.4
  500   9500 -1.802077304656480e+02 1.6e+00 1.55e+00  1e+00  2e+00 0:49.4
  600  11400 -1.802077351501054e+02 1.7e+00 1.76e+00  1e+00  2e+00 0:59.4
  700  13300 -1.802077805356563e+02 1.8e+00 2.06e+00  2e+00  2e+00 1:09.4
  800  15200 -1.802077453734133e+02 1.8e+00 2.21e+00  2e+00  2e+00 1:19.3
  900  17100 -1.802077801418513e+02 1.

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F16_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:200])
    SVM_Fun [i] = F16(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F16(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-194.48200909]), count=array([1])) 6.434602507015777
SVM
ModeResult(mode=array([-181.98427174]), count=array([1])) 4.612630624111132
ELN
ModeResult(mode=array([-193.81745135]), count=array([1])) 9.075358689496866
